# 결국엔 완성이 된거 같아서 합체 후 작업
3. requirements.csv -> 1번과 2번을 활용한 요구사항 명세서

In [2]:
import pandas as pd
import openai
import requests
import ast
import slack_alarm as alarm
import json_parsing

In [3]:
# temperature가 높게 작성을 하게 만들어서 약간 오류가 있긴함
## 해당 파일이 수정된 파일
df = pd.read_csv('data/test.csv')


In [4]:
df_test = pd.read_csv('data/total_dataset.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   subject             978 non-null    object
 1   user_input          978 non-null    object
 2   requirements        978 non-null    object
 3   total_requirements  978 non-null    object
 4   project_info        978 non-null    object
 5   ERD_data            930 non-null    object
 6   API_data            832 non-null    object
dtypes: object(7)
memory usage: 53.6+ KB


In [5]:
df_test_isnull = df_test[df_test['API_data'].isnull()].reset_index(drop=True)
df_test_isnull

,subject,user_input,requirements,total_requirements,project_info,ERD_data,API_data
0,헬시 푸드 플래너는 AI 챗봇을 통해 사용자의 건강 목표와 라이프스타일에 맞춘 식단...,"[{'projectName': '헬시 푸드 플래너', 'projectTarget':...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': '헬시 푸드 플래너...,"{'json': {'erd_tables': [{'name': 'users', 'er...",NaN
1,ClauseGuard는 AI가 계약서를 분석하여 누락된 핵심 조항이나 불공정한 내용...,"[{'projectName': 'HealthTrack AI', 'projectTar...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'HealthTra...,"{'json': {'erd_tables': [{'name': 'User', 'erd...",NaN
2,스포츠 선수 트레이드를 벤치마킹한 ‘직원 트레이드’ 플랫폼. 사내 문화·업무 부적응...,"[{'projectName': '직원 트레이드 플랫폼', 'projectTarget...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': '직원 트레이드 플...,"{'json': {'erd_tables': [{'name': 'users', 'er...",NaN
3,실시간 센서 모니터링 PyQt 앱. ESP32 기반 가스 센서 보드로부터 TCP를 ...,"[{'projectName': '스마트 농업 모니터링 시스템', 'projectTa...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': '스마트 농업 모니...,"{'json': {'erd_tables': [{'name': 'User', 'erd...",NaN
4,IoT 기반 에너지 효율 최적화 분석. 개요로는 IoT 센서 데이터를 활용하여 건물...,"[{'projectName': '스마트 에너지 분석기', 'projectTarget...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': '스마트 에너지 분...,"{'json': {'erd_tables': [{'name': '사용자', 'erd_...",NaN
...,...,...,...,...,...,...,...
141,CharGuide 서비스는 시각 장애인의 원활한 정보를 종합적으로 제공하는 웹 플랫...,"[{'projectName': 'HealthTrack AI', 'projectTar...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'HealthTra...,"{'json': {'erd_tables': [{'name': 'Users', 'er...",NaN
142,BookTrack은 독서 애호가들이 책을 효율적으로 관리하고 공유하는 문제를 해결하...,"[{'projectName': 'HealthTrack', 'projectTarget...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'HealthTra...,"{'json': {'erd_tables': [{'name': 'users', 'er...",NaN
143,플랜잇 시스템은 기업의 효과적인 연간 경영 계획 수립을 돕기 위한 통합 관리 솔루션...,"[{'projectName': 'HealthTrack Pro', 'projectTa...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'HealthTra...,"{'json': {'erd_tables': [{'name': 'Users', 'er...",NaN
144,리모트 헬스케어 플랫폼은 접근성이 낮은 의료장을 위한 원격 진료 및 건강 모니터링 ...,"[{'projectName': 'EcoTrack', 'projectTarget': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...","[{'requirementType': 'FUNCTIONAL', 'content': ...",{'json': {'project_info': {'title': 'EcoTrack'...,"{'json': {'erd_tables': [{'name': 'users', 'er...",NaN


## 978개로 데이터 생성 진행
- AWS에 있는거 활용... 을 하면 좀 그런가
- 아무도 안 쓰는거 같으니까 테스트 겸 한번 해보자
- AWS 주소 : http://13.209.5.218:8000/

In [6]:
df.columns

Index(['subject', 'user_input', 'requirements'], dtype='object')

### 진행방법
1. 요구사항 명세서 추가 생성
2. 프로젝트 요약 생성
3. ERD,API 명세서 생성

In [14]:
import re
import logging

base_url = 'http://127.0.0.1:8000/'

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def clean_escaped_quotes(text):
    """이스케이프된 따옴표를 정리하는 함수"""
    
    if not isinstance(text, str):
        return str(text)  # 문자열이 아니면 그대로 반환

    # 방법 1: 정규식으로 \' -> ' 변환
    cleaned = re.sub(r"\\'", "'", text)
    return str(cleaned)

def safe_parse_requirements(text):
    """안전하게 requirements 문자열을 파싱하는 함수"""

     # 이미 딕셔너리나 리스트인 경우 그대로 반환
    if isinstance(text, (dict, list)):
        return str(text)
    
    # 문자열이 아닌 경우 처리
    if not isinstance(text, str):
        logger.warning(f"예상치 못한 타입: {type(text)}")
        return str(text)
    
    try:
        # 1단계: 이스케이프된 따옴표 정리
        cleaned = clean_escaped_quotes(text)
        
        # 2단계: ast.literal_eval로 파싱 시도
        parsed = ast.literal_eval(cleaned)
        
        return str(parsed)
    
    except (ValueError, SyntaxError) as e:
        print(f"ast.literal_eval 실패: {e}")
        
        try:
            # 3단계: 추가 정리 후 재시도
            # 바깥쪽 따옴표 제거 (만약 있다면)
            if cleaned.startswith("'") and cleaned.endswith("'"):
                cleaned = cleaned[1:-1]
            elif cleaned.startswith('"') and cleaned.endswith('"'):
                cleaned = cleaned[1:-1]
            
            # 다시 파싱 시도
            parsed = ast.literal_eval(cleaned)
            return str(parsed)
            
        except Exception as e2:
            print(f"2차 파싱도 실패: {e2}")
            return None
    
# 요구사항 명세서 생성 함수
def generate_requirements(project_overview, requirements) :
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    requirements : 요구사항 명세서(json 형태)
    """
    # 데이터 input
    data = {
        "project_overview": project_overview,
        "existing_requirements": requirements,
        "additional_count": 5,
        "max_tokens": 4000,
        "temperature": 0.5,
        "model": "gpt-4o-mini"
    }

    # 데이터 호출
    response = requests.post(
                            url = base_url +'api/PJA/requirements/generate', 
                            json = data,
                            headers={"Content-Type": "application/json"})
    
    new_requirements = response.json()

    # 원본 데이터에 추가로 받은 데이터 추가
    add_requirements = new_requirements['requirements']
    total_requirements = str(ast.literal_eval(requirements) + add_requirements)

    return total_requirements

# 프로젝트 요약 생성 함수 
def generate_project_info(project_overview, total_requirements):
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    total_requirements : 요구사항 명세서 생성하는 코드(json 형태)
    """

    # 데이터 input
    data = {
        "project_overview": project_overview,
        "requirements": total_requirements,
        "max_tokens": 4000,
        "temperature": 0.5,
        "model": "gpt-4o-mini"
    }
    
    # 데이터 호출
    response = requests.post(
                            url = base_url +'api/PJA/json_Summury/generate', 
                            json = data,
                            headers={"Content-Type": "application/json"})
    project_info = response.json()

    return str(project_info)

# ERDAPI 생성 함수수
def generate_ERDAPI(project_overview, total_requirements, project_info) :
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    total_requirements : 요구사항 명세서 생성하는 데이터(json 형태)
    project_info : 프로젝트 요약한 데이터(json 형태태)
    """

    # 데이터 input
    data = {
            "project_overview": project_overview,
            "requirements": total_requirements,
            "project_summury": project_info,
            "max_tokens": 4000,
            "temperature": 0.8,
            "model": "ft:gpt-4o-mini-2024-07-18:test:fastapi-specialist:BjGqgQFK"
        }
    
    # 데이터 호출
    response = requests.post(
        url=base_url + 'api/PJA/json_API/generate',
        json=data,
        headers={"Content-Type": "application/json"}
    )

    if response.status_code == 200:
        ERDAPI_data = response.json()
        return ERDAPI_data, response
    else:
        return None, response  # 반드시 튜플로 반환

    # 혹시라도 위 조건문을 타지 않는 경우를 대비해 아래 한 줄 추가
    return None, None

In [15]:
import json

error_cases = []
# 테스트로 20개만 진행
df_test_isnull = df_test_isnull.iloc[:20].reset_index(drop=True)

for i in range(len(df_test_isnull)):
    a = df_test_isnull['user_input'][i]
    b = df_test_isnull['total_requirements'][i]
    c = df_test_isnull['project_info'][i]
    try:
        result, response = generate_ERDAPI(a, b, c)
        if response is not None and response.status_code == 500:
            print(f"500에러 발생! index: {i}")
            print("user_input:", a)
            print("total_requirements:", b)
            print("project_info:", c)
            # 에러 메시지 detail 출력
            try:
                error_detail = json.loads(response.text).get("detail", "")
                print("에러 detail:", error_detail)
            except Exception as e:
                print("에러 detail 파싱 실패:", e)
                print("response text:", response.text)
            print("="*40)
            error_cases.append({
                "index": i,
                "user_input": a,
                "total_requirements": b,
                "project_info": c,
                "response_text": response.text
            })
    except Exception as e:
        print("Exception 발생:", e)

500에러 발생! index: 2
user_input: [{'projectName': '직원 트레이드 플랫폼', 'projectTarget': '기업 인사팀 및 HR 전문가', 'mainFunction': ['AI 기반 인재-문화 적합도 분석', '직원 자발적 문화평 작성 기능', '트레이드 가능 기업 및 포지션 추천'], 'techStack': ['Python', 'Django', 'PostgreSQL'], 'projectDescription': '스포츠 선수 트레이드를 벤치마킹한 ‘직원 트레이드’ 플랫폼. 사내 문화·업무 부적응 인력을, 타사의 유사 스펙·성향·희망 포지션 기반 인재와 교환. 이력서·자소서·사내 후기 데이터를 AI가 분석해 트레이드 가능 기업·부서·포지션을 추천. 직원은 자발적으로 사내 문화평을 작성하고, 기업 간 조건이 맞을 경우 트레이드 성사. 채용 시 발생하는 기회비용과 조직 내 불일치 문제를 줄이며, ML/DL을 통한 인재-문화 간 적합도 매칭을 핵심 기능으로 삼음. 법적 검토 필요.'}]
total_requirements: [{'requirementType': 'FUNCTIONAL', 'content': '사용자는 자신의 이력서와 자소서를 시스템에 업로드하여 AI 분석을 통해 적합한 트레이드 가능 기업과 포지션을 추천받을 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '직원은 자발적으로 사내 문화평을 작성하고, 작성된 문화평은 다른 직원들이 열람할 수 있어야 하며, 이 평가는 트레이드 추천에 반영되어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': 'AI는 이력서, 자소서, 사내 후기 데이터를 분석하여 직원의 성향과 문화 적합도를 평가하고, 이를 기반으로 트레이드 가능성을 제시해야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 사용자 요청에 대한 응답을 2초 이내에 처리해

In [ ]:
import time

# 테스트로 10개만
project_overview_lst = df['user_input']
requirements_lst= df['requirements']

# 더미로 돌렸을때 걸리는 시간 체크도 필요할듯

# 데이터 저장해놓을 리스트
total_requirements_lst, project_info_lst, ERDAPI_data_lst = [], [], []

for project_overview, requirements, num in zip(project_overview_lst, requirements_lst, range(len(requirements_lst))) :

    # 각 코드 실행 후 리스트에 백업
    total_requirements = generate_requirements(project_overview, requirements)
    total_requirements = json_parsing.extract_json_from_response(total_requirements)
    total_requirements_lst.append(total_requirements)

    project_info = generate_project_info(project_overview, total_requirements)
    project_info = json_parsing.extract_json_from_response(project_info)
    project_info_lst.append(project_info)

    ERDAPI_data = generate_ERDAPI(project_overview, total_requirements, project_info)
    ERDAPI_data = json_parsing.extract_json_from_response(ERDAPI_data)
    ERDAPI_data_lst.append(ERDAPI_data)

    if num+1 % 50 == 0 :
        alarm.send_slack(f"{num}번째 실행 완료")

    time.sleep(1)

alarm.send_slack(f"더미데이터 생성 작업 완료")

True